In [1]:
#table 7 IE descriptive
import pandas as pd
import numpy as np
import matplotlib as plt
#!pip install linearmodels  
from linearmodels import PanelOLS
import statsmodels.discrete.discrete_model as sm

#2 econtools
#!pip install econtools
import econtools as econ
import econtools.metrics as mt

df = pd.read_stata('data/IE.dta')
df = df.sort_values(by='authority_code',ascending=True)

In [2]:
#descriptive
df['lmiles_pa_to'] = np.log(df['miles_pa_torino']+1)
del df['experience']

df = df.sort_values(by = 'authority_code', ascending = True)
exp = df['authority_code'].value_counts()
exp = pd.DataFrame({'auth':exp.index, 'value':exp.values})

for i in range(len(df)):
    for j in range(len(exp)):
        if df.loc[i,'authority_code'] == exp.loc[j, 'auth']:
            df.loc[i, 'experience'] = exp.loc[j, 'value']

df['reserve_price'] = df['reserve_price']/1000
df['population'] = df['population']/1000
df= df[['fpsb_auction', 'forcedfp_strict', 'reserve_price', 'contract_duration', 'experience', 'population', 'discount', 'n_bidders', 'miles_pa_torino', 'miles_firm_work', 'days_to_award']]
idx = df[df['forcedfp_strict']!=1].index
df = df.drop(idx)



In [4]:
#discriptive
#PanelB
describe = df.groupby('fpsb_auction')[['reserve_price', 'contract_duration', 'miles_pa_torino', 'experience', 'population', 'discount', 'miles_firm_work', 'n_bidders']].describe()
describe.head()

reserve_price                                                     \
                     count         mean          std         min          25%   
fpsb_auction                                                                    
0.0                  321.0   600.249446   636.665697   39.000000   254.000000   
1.0                   74.0  2755.674615  5169.713272  346.651563  1196.823313   

                                             contract_duration              \
                      50%     75%        max             count        mean   
fpsb_auction                                                                 
0.0            397.649375   770.0   7012.000             306.0  204.217972   
1.0           1563.211937  2542.5  43657.576              66.0  415.651520   

              ... miles_firm_work             n_bidders                        \
              ...             75%         max     count       mean        std   
fpsb_auction  ...                                                               
0.0           ...       63.790001  713.099976     321.0  66.000000  47.879078   
1.0           ...             NaN         NaN      74.0  17.554054  11.593742   

                                            
              min   25%   50%   75%    max  
fpsb_auction                                
0.0           3.0  33.0  59.0  87.0  423.0  
1.0           2.0   9.0  15.0  23.0   62.0  

[2 rows x 64 columns]

In [7]:
df = df[(df['sample']=="Telemat_MyCS_2009") | (df['sample']=="Telemat_2011") |  (df['sample']=="Telemat_2010")]
df['lmiles_pa_to'] = np.log(df['miles_pa_torino']+1)
del df['experience']

df = df.sort_values(by = 'authority_code', ascending = True)
exp = df['authority_code'].value_counts()
exp = pd.DataFrame({'auth':exp.index, 'value':exp.values})
for i in range(len(df)):
    for j in range(len(exp)):
        if df.loc[i,'authority_code'] == exp.loc[j, 'auth']:
            df.loc[i, 'experience'] = exp.loc[j, 'value']

In [8]:
auth_type_dum = pd.get_dummies(df['auth_type_region'])
auth_type_list = df['auth_type_region'].unique()
df = pd.concat([df, auth_type_dum],axis = 1)

'''year_dum = pd.get_dummies(df['year'])
year_list = df['year'].unique()
df= pd.concat([df, year_dum],axis = 1)

work_dum = pd.get_dummies(df['work_category'])
work_list = df['work_category'],unique()
df= pd.cocat([df,work_dum],axis = 1)'''

"year_dum = pd.get_dummies(df['year'])\nyear_list = df['year'].unique()\ndf= pd.concat([df, year_dum],axis = 1)\n\nwork_dum = pd.get_dummies(df['work_category'])\nwork_list = df['work_category'],unique()\ndf= pd.cocat([df,work_dum],axis = 1)"

In [31]:
reg_col = ['lmiles_pa_to', 'population', 'experience', 'municipality','logreserve_price', 'contract_duration']
for i in auth_type_list:
    reg_col.append(i)
#emilia lomagna should be omitted

In [25]:
sample = 'forcedfp_strict'
outcome = ['discount', 'days_to_award']

#for o in outcome:
df_probit = df[df[sample]==0]
probit_mod = sm.Probit(df_probit['fpsb_auction'], df_probit[reg_col],missing = 'drop')
probit_res = probit_mod.fit()
print(probit_res.summary())
#probit value checked
df_probit['proscore'] = probit_res.predict(df_probit[reg_col]) #error
df_probit.loc[(df_probit['proscore']>0.925)|(df_probit['proscore']<0.075), 'proscore'] = np.nan

Optimization terminated successfully.
         Current function value: 0.244893
         Iterations 8
                          Probit Regression Results                           
Dep. Variable:           fpsb_auction   No. Observations:                 1604
Model:                         Probit   Df Residuals:                     1593
Method:                           MLE   Df Model:                           10
Date:                Thu, 09 Jul 2020   Pseudo R-squ.:                  0.5645
Time:                        12:32:35   Log-Likelihood:                -392.81
converged:                       True   LL-Null:                       -901.99
Covariance Type:            nonrobust   LLR p-value:                2.078e-212
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
lmiles_pa_to         -0.0229      0.096     -0.238      0.812      -0.212       0.166
populati

C:\Users\thxkn\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
#OLS
'''df_reg= df[(df[sample]==0) & (df['propscore'].isnull()==False)]
reg_col = ['fpsb_auction', 'lmiles_pa_to', 'logreserve_price', 'municipality' i.auth_type_region 'population', 'experience', i.year i.work_category, 'contract_duration']
for i in auth_type_list:
    reg_col.append(i)
for j in 
fe_reg = mt.reg(df_reg, o in outcome, )'''

In [28]:
print(reg_col)

None


In [35]:
#OLS 
work_dum = pd.get_dummies(df['work_category'])
year_dum = pd.get_dummies(df['year'])
work_list = list(work_dum.columns)
year_list = list(year_dum.columns)
df_dum = pd.concat([year_dum, work_dum],axis = 1)
df = pd.concat([df, df_dum],axis = 1)

#probit reg_col에 이어서 붙이면 됨
reg_col.append('fpsb_acution')
#for i in auth_type_list:
#    reg_col.append(i)
for j in year_list:
    reg_col.append(j)
for k in work_list:
    reg_col.append(k)

if df_probit['forcedfp_strict']==0 and df_probit['proscore'].isnull()==False:
    for o in outcome:
        fe_reg_1 = mt.reg(df_probit[o],df_probit[reg_col], fe_name = 'authority_code', cluster = 'auth_anno', check_colinear = True)
    

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
for k in work_list:
    reg_col.remove(k)

if df_probit['forcedfp_strict']==0 and df_probit['proscore'].isnull()==False:
    for o in outcome:
        fe_reg_2 = mt.reg(df_probit[o],df_probit[reg_col], fe_name = 'authority_code', cluster = 'auth_anno')

In [ ]:
#Heckman IV
for k in work_list:
    reg_col.append(k)

if df_probit['forcedfp_strict']==0:
    for o in outcome:
        fe_reg_3 = mt.reg(df_probit[o],df_probit[reg_col], fe_name = 'authority_code', cluster = 'auth_anno')

#treat reg 
#skip

for i in year_list:
    reg_col.remove(i)
for k in work_list:
    reg_col.remove(k)

    
if df_probit['forcedfp_strict']==0:
fe_reg_4 = mt.reg(df_probit[o],df_probit[reg_col], fe_name = 'authority_code', cluster = 'auth_anno')

df_probit = df_probit.drop('proscore',axis = 1)

In [ ]:
#restore
#final check for loop
df = df.drop('experience',axis = 1)
df = df.sort_values(by = 'authority_code', ascending = True)
exp = df['authority_code'].value_counts()
exp = pd.DataFrame({'auth':exp.index, 'value':exp.values})
for i in range(len(df)):
    for j in range(len(exp)):
        if df.loc[i,'authority_code'] == exp.loc[j, 'auth']:
            df.loc[i, 'experience'] = exp.loc[j, 'value']

df = df[(df[sample]=="Telemat_MyCS_2009") | (df[sample]=="Telemat_2011") |  (df[sample]=="Telemat_2010")]
df['normal_base_asta'] = reserve_price/1000
df['discontinuity_rdd'] = 0
df.loc[df['reserve_price']>1000000,'discontinuity_rdd'] = 1
df[o+'_rdd'] = o
df.loc[(df['normal_base_asta']<100) & (df['fpsb_auction']==1) ,o+'_rdd'] =np.nan
df.loc[(df['authority_code']==3092130) & (df['normal_base_asta']>1000) & (df['fpsb_auction']==0), o+'_rdd'] = np.nan
df['normal_b_asta2']=df['normal_base_asta']^2
df['normal_b_asta3']=df['normal_base_asta']^3

#reg
